# AITU Project Challenge
## Кластеризация данных от Сергек

*Source: https://apc.astanait.edu.kz/sergek*

## О ПРОЕКТЕ
В данном датасете треки 1000 транспортных средств (ТС), которые катались по нашему городу в течении 1 недели ( с 7 по 13 декабря 2020) с обезличенными идентификаторами.

Одна запись - это одна детекция ТС, GPS координаты являются координатами участка дороги, где была произведена детекция.

Некоторые участки в какой-то момент времени могли не работать (потеря связи, отсутствие электропитания) или пропустить ТС (например в ракурсе камеры ТС было загорожено автобусом)

**Задача - кластеризация данных:**

1. Определить рейсовые автобусы **(3 балла)**

2. Определить такси **(7 баллов)**

3. Определить родителей, которые отвозят детей в среднюю школу **(10 баллов)**

4. Дать свою кластеризацию всех данных с описанием логики и обоснованием **(10 баллов)**


**Требования:**

Формат сдачи - ipnyb файл либо Google Colab

Количество этапов: 1

Количество баллов за этап: максимально 30 баллов

Награда: стажировка в Коркем телеком (Сергек)

Дата сдачи: до 12-го февраля

# Open tasks

1. Read about three ML tasks, regression, classification and clustering. Understand their differences. Then focus on clustering. Be ready for any questions from me about clustering. Link for sklearn clustering documentation: https://scikit-learn.org/stable/modules/clustering.html

2. Understand the data that you have. Look on the steps done below. Think how we can our data.

3. Think how we can distinguish bus, taxi and parents from each other.

4. Clarify: do we need to use an algorithm that clusters these cars or can we use data analysis for that?



# Solution

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

## 1. Data load

In [2]:
df_0 = pd.read_csv('data_sergek.csv')
df_0

,guid,datetime,latitude,longitude
0,712070869093788000,12/8/20 17:14,51.165747,71.453509
1,11686700327246500000,12/8/20 17:15,51.151115,71.426902
2,2105413917665240000,12/8/20 17:15,51.181148,71.404782
3,3462092866854250000,12/8/20 17:15,51.136498,71.415772
4,7700733881961110000,12/8/20 17:15,51.143693,71.411055
...,...,...,...,...
387017,9642416086154050000,12/13/20 11:01,51.176601,71.418422
387018,2185211928349050000,12/13/20 11:01,51.129236,71.409150
387019,403674413556996000,12/13/20 11:02,51.117594,71.401256
387020,11277932757697900000,12/13/20 11:02,51.142655,71.487506


In [3]:
df_0.shape

(387022, 4)

In [4]:
# No missing data
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387022 entries, 0 to 387021
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   guid       387022 non-null  uint64 
 1   datetime   387022 non-null  object 
 2   latitude   387022 non-null  float64
 3   longitude  387022 non-null  float64
dtypes: float64(2), object(1), uint64(1)
memory usage: 11.8+ MB


In [5]:
# We have 1000 unique guid, i.e. 1000 cars in our dataset
len(df_0.guid.unique())

1000

In [6]:
# Observe latitude and longitude
df_0[['latitude', 'longitude']].describe()

,latitude,longitude
count,387022.000000,387022.000000
mean,51.146520,71.435415
std,0.026620,0.038694
min,51.030354,71.252515
25%,51.129842,71.410049
50%,51.148984,71.428700
75%,51.166068,71.463849
max,51.212532,71.648309


In [7]:
df_0['coordinates'] = df_0['latitude'].astype(str) + ' - ' + df_0['longitude'].astype(str)
df_0.head()

,guid,datetime,latitude,longitude,coordinates
0,712070869093788000,12/8/20 17:14,51.165747,71.453509,51.16574734 - 71.45350872
1,11686700327246500000,12/8/20 17:15,51.151115,71.426902,51.15111466 - 71.42690215
2,2105413917665240000,12/8/20 17:15,51.181148,71.404782,51.1811477 - 71.40478196
3,3462092866854250000,12/8/20 17:15,51.136498,71.415772,51.13649764 - 71.41577178
4,7700733881961110000,12/8/20 17:15,51.143693,71.411055,51.14369334 - 71.41105484


In [8]:
# Based on the unique values of latitude and longitude we can say that there are 645 cameras in the city
# To make sure that we also created new columns coordinates (location of each camera), which tells us that
# there are indeed 645 unique coordinates of cameras
print(len(df_0.latitude.unique()))
print(len(df_0.longitude.unique()))
print(len(df_0.coordinates.unique()))

645
645
645


Our data for lattitude and longitude is centered around 51 and 71, respectively. This is actually coordinates for Nur-Sultan. Check: https://en.wikipedia.org/wiki/Nur-Sultan. Hence, we will need to find a way to zoom into Nur-Sultan only and focus on these small area. 

In [ ]:
# Create new columns based on datetime column
df_0['datetime'] =  pd.to_datetime(df_0['datetime']) # convert to datetime format 
df_0['date'] = df_0['datetime'].dt.date              # get column only for date
df_0['time'] = df_0['datetime'].dt.strftime('%H:%M') # get column only for time
df_0['weekday'] = df_0['datetime'].dt.day_name()      # get column only for weekday
#df_0['year'] = df_0['datetime'].dt.year              # get column only for year
#df_0['month'] = df_0['datetime'].dt.month            # get column only for month
#df_0['day'] = df_0['datetime'].dt.day                # get column only for day
df_0['hour'] = df_0['datetime'].dt.hour              # get column only for hour
#df_0['minute'] = df_0['datetime'].dt.minute          # get column only for minute
df_0.head()

In [ ]:
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
from vega_datasets import data as vds

In [ ]:
# Plot map of all cameras with activated cameras for a given car on a given day
sns.set_style('whitegrid')
plt.figure(figsize=(20,15))

# Define the borders for lattitude and longitude (i.e. Nur-Sultan city)
plt.xlim(min(df_0.latitude) - 0.01, max(df_0.latitude) + 0.01)
plt.ylim(min(df_0.longitude) - 0.01,max(df_0.longitude) + 0.01)

# Plot all the cameras on the map
sns.scatterplot(x='latitude', y='longitude', data=df_0, 
                marker='o', linewidth=0, color='lightgrey', s=15)

In [ ]:
# Create dataframe with unique coordinates of cameras
df_sergeks = df_0[['coordinates', 'latitude', 'longitude']]
df_sergeks.drop_duplicates(inplace=True)
df_sergeks

In [ ]:
df_1 = df_0[['weekday', 'guid', 'coordinates', 'time']]\
            .groupby(['weekday', 'guid', 'coordinates'], as_index=False).count()

df_1 = df_1.merge(df_sergeks, on='coordinates', how='left')

df_1

In [ ]:
# Plot map of all cameras with activated cameras for a given car
sns.set_style('whitegrid')
plt.figure(figsize=(20,15))

# Define the borders for lattitude and longitude (i.e. Nur-Sultan city)
plt.ylim(min(df_0.latitude) - 0.01, max(df_0.latitude) + 0.01)
plt.xlim(min(df_0.longitude) - 0.01,max(df_0.longitude) + 0.01)

# Plot all the cameras on the map
sns.scatterplot(x='longitude', y='latitude', data=df_0, 
                marker='o', linewidth=0, color='lightgrey', s=15)

#Plot the cameras on which car appeared: specify day and guid
sns.scatterplot(x='longitude', y='latitude', data=df_1[(df_1.guid==4932614453064340) & (df_1.weekday=='Friday')],
                hue='weekday', palette='tab10', 
                marker='s', linewidth=0, s=50);

In [ ]:
check = df_1[(df_1['guid'] == 4932614453064340) & (df_1['weekday'] == 'Friday')]

sergek = folium.Map(location=[51.1605, 71.4104], zoom_start=11.5)
check.apply(lambda row: folium.Marker(location=[row['latitude'], row['longitude']],
                                    popup=row['weekday']).add_to(sergek), axis=1)

sergek

In [ ]:
# Plot map of all cameras with activated cameras for a given car
sns.set_style('whitegrid')
plt.figure(figsize=(20,15))

# Define the borders for lattitude and longitude (i.e. Nur-Sultan city)
plt.ylim(min(df_0.latitude) - 0.01, max(df_0.latitude) + 0.01)
plt.xlim(min(df_0.longitude) - 0.01,max(df_0.longitude) + 0.01)

# Plot all the cameras on the map
sns.scatterplot(x='longitude', y='latitude', data=df_0, 
                marker='o', linewidth=0, color='lightgrey', s=15)

#Plot the cameras on which car appeared: specify day and guid
sns.scatterplot(x='longitude', y='latitude', data=df_1[df_1.guid==18445985492879400000],
                hue='weekday', palette='tab10', 
                marker='s', linewidth=0, s=50);

In [ ]:
check = df_1[df_1['guid'] == 18445985492879400000]

sergek = folium.Map(location=[51.1605, 71.4104], zoom_start=11.5)
check.apply(lambda row: folium.Marker(location=[row['latitude'], row['longitude']],
                                    popup=row['weekday']).add_to(sergek), axis=1)

sergek

It is not really helpful to use datetime column in it's original form. But, we can split it into separate peaces of data: date, time, weekday, year, month, day, hour and minute. However, columns year and month are not really usefull, because our data only for 7-13 December of 2020, i.e. only one month and one year.

## 2. Analysis

Our task is to identify which guids correspond to buses, cars and parent drivers. We should think about the patterns of driving of each category. Below are some ideas:
1. Bus drivers are the simplest, because they have certain route (маршрут). To find that, you can look on a car's movements, how it is recorded on the camera.
2. Taxi drivers are more difficult, because they can be absolutely chaotic in their driving: no certain path, no time periodicity.
3. Parent drivers is more difficult than buses, but easier than drivers. We can expect that they will have fixed route (i.e. the same camera coordinates), and some time constraints (с дома до школы/работы утром и вечером с работы/дома до школы). 

You can brainstorm yourself on these patterns. But still, I want to clarify, if we need to build a ML model for that or just use analysis.

In [ ]:
df_0.head()

In [ ]:
# Create dataframe with features of the cars. Start with unique guids.
df_features = df_0[['guid']]
df_features.drop_duplicates(inplace=True)
df_features.head()

In [ ]:
# So we have 1000 unique cars
df_features.shape

#### Feature 1
Total number of fixations in one week.

In [ ]:
df_feat_1 = df_0[['guid', 'datetime']].groupby('guid', as_index=False).count()
df_feat_1.rename(columns={"datetime": "Total_nb_fixations"}, inplace=True)
df_features = df_features.merge(df_feat_1, on='guid', how='left')
df_features.head()

#### Feature 2
Average number of fixations in per day.

In [ ]:
df_feat_2 = df_0[['guid', 'weekday', 'datetime']].groupby(['guid','weekday'], as_index=False).count() 
df_feat_2_ = df_feat_2.groupby('guid',  as_index=False).mean()
df_feat_2_.rename(columns={"datetime": "Avg_nb_fixations_per_day"}, inplace=True)
df_features = df_features.merge(df_feat_2_, on='guid', how='left')
df_features.head()

#### Feature 3
Number of active days in a week.

In [ ]:
df_feat_3 = df_0[['guid', 'weekday']]
df_feat_3.drop_duplicates(inplace=True)
df_feat_3_ = df_feat_3.groupby('guid',  as_index=False).count()
df_feat_3_.head()
df_feat_3_.rename(columns={"weekday": "Nb_active_days"}, inplace=True)
df_features = df_features.merge(df_feat_3_, on='guid', how='left')
df_features.head()

#### Feature 4
Number of unique sergeks detected. 

In [ ]:
df_feat_4 = df_0[['guid', 'coordinates']]
df_feat_4.drop_duplicates(inplace=True)
df_feat_4_ = df_feat_4.groupby('guid',  as_index=False).count()
df_feat_4_.head()
df_feat_4_.rename(columns={"coordinates": "Nb_sergeks"}, inplace=True)
df_features = df_features.merge(df_feat_4_, on='guid', how='left')
df_features.head()

#### Feature 5
Average number of unique sergeks per day

In [ ]:
df_feat_5 = df_0[['guid', 'weekday', 'coordinates']]
df_feat_5.drop_duplicates(inplace=True)
df_feat_5_ = df_feat_5.groupby(['guid', 'weekday'], as_index=False).count()
df_feat_5_ = df_feat_5_.groupby(['guid'], as_index=False).mean()
df_feat_5_.rename(columns = {'coordinates' : 'Avg_nb_unique_sergeks_per_day'}, inplace=True)
df_features = df_features.merge(df_feat_5_, on='guid', how='left')
df_features.head()

#### Feature 6
Average number of working hours per day

In [ ]:
df_feat_6 = df_0[['guid', 'weekday', 'hour']]
df_feat_6.drop_duplicates(inplace = True)
df_feat_6_ = df_feat_6.groupby(['guid', 'weekday'], as_index=False).count()
df_feat_6_ = df_feat_6_.groupby(['guid'], as_index=False).mean()
df_feat_6_.rename(columns = {'hour':'Avg_nb_hours_per_day'}, inplace=True)
df_features = df_features.merge(df_feat_6_, on='guid', how='left') 
df_features.head()

#### Feature 7
Average the most latest hour per day

In [ ]:
df_feat_7 = df_0[['guid', 'weekday', 'hour']]
df_feat_7.drop_duplicates(inplace=True)
df_feat_7_ = df_feat_7.groupby(['guid', 'weekday'], as_index=False).max()
df_feat_7_ = df_feat_7_.groupby(['guid'], as_index=False).mean()
df_feat_7_.rename(columns={'hour' : 'Avg_the_most_latest_hour_per_day'}, inplace=True)
df_features = df_features.merge(df_feat_7_, on='guid', how='left')

#### Feature 8

In [ ]:
df_feat_8 = df_0[['guid', 'weekday', 'hour']]
df_feat_8.drop_duplicates(inplace=True)
df_feat_8_ = df_feat_8.groupby(['guid', 'weekday'], as_index=False).min()
df_feat_8_ = df_feat_8_.groupby(['guid'], as_index=False).mean()
df_feat_8_.rename(columns={'hour' : 'Avg_the_most_earliest_hour_per_day'}, inplace=True)
df_features = df_features.merge(df_feat_8_, on='guid', how='left')

## Observe relationship

## Example: Cluster with k-means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
model_kmeans = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans.fit(df_features[['Nb_sergeks', 'Avg_nb_fixations_per_day']])
model_kmeans.cluster_centers_
df_features['Cluster'] = model_kmeans.labels_

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Nb_sergeks', y='Avg_nb_fixations_per_day', data=df_features, 
                hue='Cluster', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans1 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans1.fit(df_features[['Nb_sergeks', 'Total_nb_fixations']])
model_kmeans1.cluster_centers_
df_features['Cluster1'] = model_kmeans1.labels_
df_features.head()

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Nb_sergeks', y='Total_nb_fixations', data=df_features, 
                hue='Cluster1', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans2 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans2.fit(df_features[['Nb_active_days', 'Total_nb_fixations']])
model_kmeans2.cluster_centers_
df_features['Cluster2'] = model_kmeans2.labels_

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Nb_active_days', y='Total_nb_fixations', data=df_features, 
                hue='Cluster2', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans3 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans3.fit(df_features[['Nb_sergeks', 'Avg_nb_unique_sergeks_per_day']])
model_kmeans3.cluster_centers_
df_features['Cluster3'] = model_kmeans3.labels_
df_features.head()

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Nb_sergeks', y='Avg_nb_unique_sergeks_per_day', data=df_features, 
                hue='Cluster3', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans4 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans4.fit(df_features[['Nb_active_days', 'Avg_nb_unique_sergeks_per_day']])
model_kmeans4.cluster_centers_
df_features['Cluster4'] = model_kmeans4.labels_

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Nb_active_days', y='Avg_nb_unique_sergeks_per_day', data=df_features, 
                hue='Cluster4', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans5 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans5.fit(df_features[['Avg_nb_hours_per_day', 'Avg_nb_unique_sergeks_per_day']])
model_kmeans5.cluster_centers_
df_features['Cluster5'] = model_kmeans5.labels_

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Avg_nb_hours_per_day', y='Avg_nb_unique_sergeks_per_day', data=df_features, 
                hue='Cluster5', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans6 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans6.fit(df_features[['Avg_the_most_latest_hour_per_day', 'Avg_the_most_earliest_hour_per_day']])
model_kmeans6.cluster_centers_
df_features['Cluster6'] = model_kmeans6.labels_

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Avg_the_most_latest_hour_per_day', y='Avg_the_most_earliest_hour_per_day', data=df_features, 
                hue='Cluster6', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans7 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans7.fit(df_features[['Nb_sergeks', 'Avg_nb_hours_per_day']])
model_kmeans7.cluster_centers_
df_features['Cluster7'] = model_kmeans7.labels_

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Nb_sergeks', y='Avg_nb_hours_per_day', data=df_features, 
                hue='Cluster7', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
model_kmeans8 = KMeans(init="random", n_clusters=3, random_state=42)
model_kmeans8.fit(df_features[['Nb_sergeks', 'Avg_nb_hours_per_day']])
model_kmeans8.cluster_centers_
df_features['Cluster8'] = model_kmeans8.labels_

sns.set_style('whitegrid')
plt.figure(figsize=(10,10))

sns.scatterplot(x='Avg_nb_hours_per_day', y='Avg_the_most_earliest_hour_per_day', data=df_features, 
                hue='Cluster8', marker='o', palette='tab10', linewidth=0, s=15);

In [ ]:
# Avg_the_most_latest_hour_per_Day and Avg_the_most_earlies_hour_per_day
df_features['Cluster6'].replace({0: "parent", 1: "taxi driver", 2:"bus driver"}, inplace=True)

In [ ]:
# Avg_ng_hours_per_day and Avg_nb_unique_sergeks_per_day
df_features['Cluster5'].replace({0: "taxi driver", 1: "bus driver", 2:"parent"}, inplace=True)

In [ ]:
# Nb_Sergeks and Avg_nb_fixations_per_day
df_features['Cluster'].replace({0: "bus driver", 1: "parent", 2:"taxi driver"}, inplace=True)

In [ ]:
# Nb_Sergeks and Avg_nb_hours_per_day
df_features['Cluster7'].replace({0: "parents", 1: "taxi driver", 2:"parent"}, inplace=True)

In [ ]:
df_features

In [ ]:
df_features = df_features.drop('Cluster7', axis = 1)

## Conclusion
In general, we had 8 clustering models by experimenting with different variables. On 6th Clustering Model, we tried with average the earliest and average the most latest hour. I think this correlation of variables is the best one. Below I am trying to explain this.
- taxi drivers should have the largest number of unique sergeks, because they go "randomly" around the city, buses have a little less, and parents have the smallest number of sergeks.
- parents have the least number of hours on the road, taxi drivers by a large margin should have more than their parents, but perhaps less than bus drivers, or not, I'm not sure about that.
-the early hour needs to be bus drivers, because they 6-7 am already go, then there are parents who consistently 7-8 in the morning drive around the city, and the third place among taxi drivers, they also get up early, but sometimes and in the afternoon to start work therefore their average early hour should be more than those two groups.
-the late hour at bus drivers, their working day ends to the 11-12 o'clock in the morning, followed by taxi drivers because many taxi drivers go in the evening, but sometimes they can return home earlier, and the third place of the parents, their working day ends for 5-7 hours.